In [1]:
# -*- coding: utf-8 -*-
"""
Script de Previsão de Demanda para Gerenciamento de Estoque.

Este script realiza as seguintes etapas:
1.  Gera um arquivo de dados de vendas fictício ('vendas.csv').
2.  Carrega e pré-processa os dados.
3.  Divide os dados em conjuntos de treino e teste.
4.  Treina e compara dois modelos de regressão: Regressão Linear e Random Forest.
5.  Implementa técnicas de interpretabilidade para o melhor modelo.
6.  Avalia o desempenho dos modelos usando o Erro Absoluto Médio (MAE).
7.  Usa o melhor modelo para prever a demanda para os próximos 7 dias.
8.  (Bônus) Visualiza a demanda real vs. prevista.
9.  (Bônus) Permite a previsão para um SKU específico fornecido pelo usuário.
"""

"\nScript de Previsão de Demanda para Gerenciamento de Estoque.\n\nEste script realiza as seguintes etapas:\n1.  Gera um arquivo de dados de vendas fictício ('vendas.csv').\n2.  Carrega e pré-processa os dados.\n3.  Divide os dados em conjuntos de treino e teste.\n4.  Treina e compara dois modelos de regressão: Regressão Linear e Random Forest.\n5.  Implementa técnicas de interpretabilidade para o melhor modelo.\n6.  Avalia o desempenho dos modelos usando o Erro Absoluto Médio (MAE).\n7.  Usa o melhor modelo para prever a demanda para os próximos 7 dias.\n8.  (Bônus) Visualiza a demanda real vs. prevista.\n9.  (Bônus) Permite a previsão para um SKU específico fornecido pelo usuário.\n"

In [2]:
# --- ETAPA 0: Importação de Bibliotecas  ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance
import os

In [3]:
# --- ETAPA 1: Carregamento dos Dados ---
print("\n--- ETAPA 1: Carregando os Dados ---")
try:
    dados = pd.read_csv('vendas.csv', parse_dates=['data_venda'])
    print("Dados carregados com sucesso.")
    print("Primeiras 5 linhas dos dados:")
    print(dados.head())
    print("\nInformações do DataFrame:")
    dados.info()
except FileNotFoundError:
    print("ERRO: Arquivo 'vendas.csv' não encontrado. Execute a geração de dados primeiro.")
    exit()


--- ETAPA 1: Carregando os Dados ---
Dados carregados com sucesso.
Primeiras 5 linhas dos dados:
   sku data_venda  venda
0  336 2024-01-14     15
1  336 2024-01-18     28
2  336 2024-01-20     34
3  336 2024-01-23     32
4  336 2024-01-25     27

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135226 entries, 0 to 135225
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   sku         135226 non-null  int64         
 1   data_venda  135226 non-null  datetime64[ns]
 2   venda       135226 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 3.1 MB


In [4]:
# --- ETAPA 2: Pré-processamento e Engenharia de Atributos ---
print("\n--- ETAPA 2: Pré-processamento e Engenharia de Atributos ---")

def preprocessar_dados(df):
    """Cria novos atributos a partir da data para o modelo."""
    df_proc = df.copy()
    df_proc['dia_da_semana'] = df_proc['data_venda'].dt.dayofweek
    df_proc['dia_do_mes'] = df_proc['data_venda'].dt.day
    df_proc['mes'] = df_proc['data_venda'].dt.month
    df_proc['ano'] = df_proc['data_venda'].dt.year
    df_proc['semana_do_ano'] = df_proc['data_venda'].dt.isocalendar().week.astype(int)
    
    # Lag feature: vendas da semana anterior (importante para capturar sazonalidade semanal)
    df_proc['vendas_lag_7'] = df_proc.groupby('sku')['venda'].shift(7)
    
    # Lidando com valores ausentes criados pelo lag
    df_proc = df_proc.dropna()

    # One-Hot Encoding para a variável categórica 'sku'
    df_proc = pd.get_dummies(df_proc, columns=['sku'], drop_first=True)
    
    return df_proc

dados_proc = preprocessar_dados(dados)
print("Dados pré-processados. Novos atributos criados.")
print("Exemplo de dados processados:")
print(dados_proc.head())


--- ETAPA 2: Pré-processamento e Engenharia de Atributos ---
Dados pré-processados. Novos atributos criados.
Exemplo de dados processados:
   data_venda  venda  dia_da_semana  dia_do_mes  mes   ano  semana_do_ano  \
7  2024-01-28     16              6          28    1  2024              4   
8  2024-01-31     27              2          31    1  2024              5   
9  2024-02-01     28              3           1    2  2024              5   
10 2024-02-02     24              4           2    2  2024              5   
11 2024-02-03     56              5           3    2  2024              5   

    vendas_lag_7  sku_2  sku_3  ...  sku_5666  sku_5667  sku_5668  sku_5669  \
7           15.0  False  False  ...     False     False     False     False   
8           28.0  False  False  ...     False     False     False     False   
9           34.0  False  False  ...     False     False     False     False   
10          32.0  False  False  ...     False     False     False     False   
11

In [5]:
# --- ETAPA 3: Divisão dos Dados em Treinamento e Teste ---
print("\n--- ETAPA 3: Divisão dos Dados ---")
# Para séries temporais, a divisão não deve ser aleatória.
# Vamos usar al últimas 3 semanas como conjunto de teste.
dados_proc = dados_proc.set_index('data_venda')
data_corte = dados_proc.index.max() - pd.DateOffset(weeks=3)

treino = dados_proc[dados_proc.index <= data_corte]
teste = dados_proc[dados_proc.index > data_corte]

X_treino = treino.drop('venda', axis=1)
y_treino = treino['venda']
X_teste = teste.drop('venda', axis=1)
y_teste = teste['venda']

print(f"Dados divididos em treino e teste.")
print(f"Tamanho do conjunto de treino: {len(X_treino)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_teste)} amostras")
print(f"Data de corte para divisão: {data_corte.date()}")


--- ETAPA 3: Divisão dos Dados ---
Dados divididos em treino e teste.
Tamanho do conjunto de treino: 77494 amostras
Tamanho do conjunto de teste: 27147 amostras
Data de corte para divisão: 2024-03-18


In [6]:
# --- ETAPA 4: Treinamento e Comparação de Modelos ---
print("\n--- ETAPA 4: Treinamento e Comparação de Modelos ---")
modelos = {
    "Regressão Linear": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

for nome, modelo in modelos.items():
    print(f"Treinando modelo: {nome}...")
    modelo.fit(X_treino, y_treino)
    print(f"{nome} treinado.")


--- ETAPA 4: Treinamento e Comparação de Modelos ---
Treinando modelo: Regressão Linear...
Regressão Linear treinado.
Treinando modelo: Random Forest...
Random Forest treinado.
